In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
import pandas as pd
from abc import ABC, abstractmethod

from lib.template.land_to_raw import *

In [0]:
rundate = '20130208_000000'
base_directory = f'/Workspace/Users/armando.n90@gmail.com/users_case/lakehouse/landing/visitas/{rundate}/'

for filename in os.listdir(base_directory):
    path = os.path.join(base_directory, filename)
    
    if os.path.isfile(path):

        #TO DO - PREVENT FOR REPROCESSING
        elements = filename.split('/')
        batch_id = elements[-1].split('.')[0].split('_')[1]

        table_name = 'raw_visitas'
        domain = 'domain'
        environment = 'dev'

        schema_name = 'bronze_analytics'
        catalog_name = domain + '_' + environment

        print('Processing ' + table_name + ' with batch ' + batch_id)
        factory_land_to_raw = FactoryLandToRawFromCsv(path)

        #path = f"dbfs:/Workspace/Users/armando.n90@gmail.com/databricks_case/lakehouse/landing/healthsys/{batch_id}/certificates.csv"


        factory_land_to_raw.create()

        table_processor = LandToRawTemplate(catalog_name=catalog_name, schema_name=schema_name, table_name=table_name, batch_id=batch_id)  
        table_processor.set_component_factory(factory_land_to_raw) 
        table_processor.process()

        print('')
        print(table_processor.metadata)
        print(table_processor.metrics)
        print('')
        table_processor.dataframe.show(3)
        print('')
        break

In [0]:
table = 'domain_dev.bronze_analytics.raw_visitas'
test = spark.read.table(table)
test.printSchema()
test.show(5)


In [0]:
test = spark.read.table('governance_prod.metrics.ingestions')
print(test.count())
test.show(20)

In [0]:
spark.sql('DROP TABLE IF EXISTS domain_dev.bronze_analytics.raw_visitas')
spark.sql('DELETE FROM governance_prod.metrics.ingestions')